# 샌프란시스코 범죄 분류 대회

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
pd.options.display.max_columns =100

In [ ]:
train = pd.read_csv('/kaggle/input/sf-crime/train.csv.zip',parse_dates=['Dates'])
# train['year'] = train['Dates'].dt.year
# train['month'] = train['Dates'].dt.month
# train['day'] = train['Dates'].dt.day
# train['hour'] = train['Dates'].dt.hour
train.head()

In [ ]:
# display(train['year'].unique(),train['month'].unique(),train['day'].unique(),train['hour'].unique())

In [ ]:
# display(test['year'].unique(),test['month'].unique(),test['day'].unique(),test['hour'].unique())

In [ ]:
test = pd.read_csv('/kaggle/input/sf-crime/test.csv.zip',parse_dates=['Dates'])
# test['year'] = test['Dates'].dt.year
# test['month'] = test['Dates'].dt.month
# test['day'] = test['Dates'].dt.day
# test['hour'] = test['Dates'].dt.hour
test.head()

In [ ]:
display(train.shape,test.shape)

In [ ]:
train['Category'].unique() # Target Variable

In [ ]:
display(train['Resolution'].unique(), train['Resolution'].nunique()) # Only in train.csv

In [ ]:
# display(train['Descript'].unique()[:50],train['Descript'].nunique()) # Only in train.csv

## Base Line

In [ ]:
display(train.isnull().any(),test.isnull().any())

* No NA

In [ ]:
alldata = pd.concat([train,test])
alldata.head()

In [ ]:
alldata2 = alldata.drop(['Category','Resolution','Descript','Dates','Id'],axis=1)
alldata2.head()

In [ ]:
train['Category'].value_counts()

* 정답값이 어떻게 분포되어 있는지 확인하는것 매우 중요.

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

for i in alldata2.columns[alldata2.dtypes==object]:
    alldata2[i] = le.fit_transform(alldata2[i])

In [ ]:
alldata2.head()

In [ ]:
train2 = alldata2[:len(train)]
test2 = alldata2[len(train):]

# Creating evaluation set

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_valid,y_train,y_valid = train_test_split(train2,train['Category'],test_size=0.2,random_state=1,stratify=train['Category'])
y_valid.value_counts()

* random_state 고정 필수.
* stratify=train['Category'] : train의 목적변수 각각의 비율대로 뽑겠다는 의미.

In [ ]:
%%time
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(random_state=0, max_depth=6, n_jobs=-1)
rf.fit(x_train,y_train)

In [ ]:
valid_result = rf.predict_proba(x_valid)

In [ ]:
from sklearn.metrics import log_loss
log_loss(y_valid, valid_result)

In [ ]:
%%time
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(random_state=0, max_depth=6, n_jobs=-1)
rf.fit(train2,train['Category'])

* 랜덤포레스트하면 램 터져서 안돌아감.
* max_depth = -1 이 default. 지금 데이터셋 매우 복잡. 하나의 나무에서 너무 많은 학습을 해버림. 나무 깊이를 설정하자.

In [ ]:
result = rf.predict_proba(test2)

In [ ]:
sub=pd.read_csv('/kaggle/input/sf-crime/sampleSubmission.csv.zip')
sub.head()

In [ ]:
sub.shape #Id 제외

In [ ]:
len(result[0])

In [ ]:
result.shape

In [ ]:
#sub.iloc[행,열]
sub.iloc[:,1:]

* 칼럼 39개에 접근해서 값 넣어준다.

In [ ]:
sub.iloc[:,1:] = result
sub.head()

In [ ]:
sub.to_csv('submission.csv',index=False)